In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import csv
import requests
import time
import os
import json

from tqdm import tqdm

In [ ]:
# configure
file_path = '../references/questions.txt'
output_path = '../data/exports'
delay_seconds = 5

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
list_of_questions = []

with open(file_path, 'rt') as txt_file:
  for text in txt_file:
    text = text.strip()
    if text != '':
        list_of_questions.append(text)

In [ ]:
file_data = {}
file_extra = {}

for question in tqdm(list_of_questions):
    list_data = []
    list_data_extra = []
    time.sleep(delay_seconds)
    response = requests.get('https://scripturecentralqa.org/api/search', params={'q': question})
    if response.status_code != 200:
        print(f"ERROR on query {question}: status_code {response.status_code}")
        continue
    try:
        res = response.content.decode('utf-8')
        result_data = json.loads(res)
    except Exception as e:
        print(f"ERROR on query {question} parsing response {res}")
        continue

    time.sleep(delay_seconds)
    gpt_response = requests.get('https://scripturecentralqa.org/api/search', params={'q': question, 'query_type': 'norag'})
    if gpt_response.status_code != 200:
        print(f"ERROR on query {question}: status_code {gpt_response.status_code}")
        continue
    try:
        res = gpt_response.content.decode('utf-8')
        gpt_result = json.loads(res)
    except Exception as e:
        print(f"ERROR on gpt query {question} parsing response {res}")
        continue

    for item in result_data['results']:
        result_temp = json.dumps(item)
        list_data.append(result_temp)
        if len(list_data) > 4:
            break

    for item in result_data['results']:
        result_temp = json.dumps(item)
        list_data_extra.append(result_temp)
        if len(list_data_extra) > 19:
            break

    result_set = '\n--------\n'.join(list_data)

    file_data[question] = [result_set, result_data['answer'], gpt_result["answer"]]
    file_extra[question] = [list_data_extra, result_data['answer']]

In [ ]:
with open(output_path + '/question-answers.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Question', 'Results', 'Answer', 'ChatGPT Answer'])  # header row
    for key,value in file_data.items():
        # for each question, the file should contain a line like this:
        writer.writerow([key, value[0], value[1], value[2]])

In [ ]:
with open(output_path + '/rank-question-answers.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Question', 'ResultRank', 'ResultURL', 'ResultTitle', 'ResultText'])  # header row
    for key,value in file_extra.items():
        for val in value[0]:
            value_res = json.loads(val)
            # for each question, the file should contain a line like this:
            writer.writerow([key, value_res["index"], value_res["url"], value_res["title"], value_res["text"]])

print('End')